# Functions and run the Y calling. Eventually put into Python package

In [19]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp

# For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
# Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/y_chrom/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-235.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/y_chrom
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


In [20]:
################################################
# Definitions to load Files (ISOGG and Output of Pulldown)

def create_haplopath(haplogroup, rest="_data.csv", haplo_folder="./data/isogg_19/"):
    """Give back the full Path of Haplogroup data"""
    assert(len(haplogroup) == 1)
    path = haplo_folder + haplogroup + rest
    return path

def load_haplogroup_data(path, build="Build 37 #"):
    # Change path here depenging on what haplogroup!!!!
    print("Loading: %s" % path)
    df_g = pd.read_csv(path, index_col=False, sep=",")
    df_g = df_g.rename(columns={build: 'pos'})  # For merging
    l = len(df_g)
    df_g = df_g[pd.to_numeric(df_g['pos'], errors='coerce').notnull()]
    df_g['pos'] = df_g['pos'].apply(pd.to_numeric)
    print("Drop from %i to %i numerical pos rows" % (l, len(df_g)))
    return df_g

def load_counts(path_counts, coerce=True):
    """Load Count file and return Dataframe"""
    df_t = pd.read_csv(path_counts, header=None, delim_whitespace=True)
    df_t.columns = ["snp", "chr", "pos", "ref_all", "alt_all","drop","iid","ref", "alt"]
    
    if coerce:
        for col in ["pos", "ref", "alt"]:
            df_t[col] = pd.to_numeric(df_t[col], errors="coerce")
            
    df_t = df_t.drop(columns="drop")
    return df_t

################################################
################################################
# Functions to call Y

def call_ind(df_counts, df_isogg, individual="", pos=[], 
             full=False, output=True):
    """Check individual against all markers in df_marker.
    Return dataframe with calls and 3 additional summary stats
    derived: Array whether ancestral allele is coded as dervied. (for r1b)
    individual: For which Individual to do the call (if none given use all rows).
    pos: List of specific SNPs for which to return calls (if none return all)"""

    ### Filter raws with Individual!
    if len(individual)>0:
        df_counts = df_counts[df_counts["Individual"]==individual]
        
    if output:
        print(f"Total rows of calls: {len(df_counts)}")
    
    ### Merge the two dataframes on identical positions
    df = pd.merge(df_counts, df_isogg, on="pos")
    df.drop_duplicates(subset="pos", keep='first', inplace=True)
    
    flip =  df["alt_all"]==df["Mutation info"].str[0]
    df.loc[flip,["ref", "alt", "ref_all", "alt_all"]]= df[["alt","ref", "alt_all", "ref_all"]][flip].values
        
    ### Extract only raws with more Alts than Refs
    derived = (df["alt"]>df["ref"]) & (df["alt"]>0)  # Derived Calls
    tot_calls =  (df["ref"]>0) | (df["alt"]>0)       # At least one Calls
    
    if output==True:
        print(f"Markers found in Database {len(df)}")
        print(f"Flipping {np.sum(flip)} SNPs for Alt Allele")
        print(f"Markers witch calls {np.sum(tot_calls)}")
        print(f"Frac. of Database SNPs covered: {np.sum(tot_calls) / len(df):.6f}")
        print(f"Markers derived: {np.sum(derived)}")
    
    if full == False:    # Only call the derived Markers
        df_call = df[derived]
    
    elif full == True:   # Report the whole Status
        df_call = df
    
    df_call = df_call.sort_values(by="Haplogroup")
    
    ## Only return selected Positions if needed
    if len(pos)>0:
        idx = df_call["pos"].isin(pos)
        df_call = df_call[idx]
    
    return df_call.copy(), np.sum(derived), np.sum(tot_calls), len(df)

################################################
################################################
# Functions to plot Y calls

def pull_clade(df, haplogroup, exact=True):
    """Pulls a specific clade. 
    if exact=True: Has to match the exact haplogroup"""
    if exact:
        df_t = df[df["Haplogroup"]==haplogroup]
    else:
        df_t = df[df["Haplogroup"].str.contains(haplogroup)]
    return df_t

### Load ISOGG Reference Dictionary

In [21]:
haplogroups = ["o", "d", "g", "i", "r", "j",
               "t", "e", "h", "c", "l", "n", "p", "q"]
haplo_paths = [create_haplopath(
    x, haplo_folder="./data/isogg_19/") for x in haplogroups]
df_groups = [load_haplogroup_data(p) for p in haplo_paths]

dict_y_dfs = dict(zip(haplogroups, df_groups))   # Create Dictionary
# dict_y_dfs['all'] = pd.concat(df_groups)   # Add the combined one

Loading: ./data/isogg_19/o_data.csv
Drop from 3459 to 3455 numerical pos rows
Loading: ./data/isogg_19/d_data.csv
Drop from 2595 to 2586 numerical pos rows
Loading: ./data/isogg_19/g_data.csv
Drop from 7503 to 7492 numerical pos rows
Loading: ./data/isogg_19/i_data.csv
Drop from 11246 to 11231 numerical pos rows
Loading: ./data/isogg_19/r_data.csv
Drop from 9763 to 9726 numerical pos rows
Loading: ./data/isogg_19/j_data.csv
Drop from 5947 to 5940 numerical pos rows
Loading: ./data/isogg_19/t_data.csv
Drop from 382 to 381 numerical pos rows
Loading: ./data/isogg_19/e_data.csv
Drop from 10478 to 10459 numerical pos rows
Loading: ./data/isogg_19/h_data.csv
Drop from 3140 to 3127 numerical pos rows
Loading: ./data/isogg_19/c_data.csv
Drop from 6862 to 6855 numerical pos rows
Loading: ./data/isogg_19/l_data.csv
Drop from 1154 to 1151 numerical pos rows
Loading: ./data/isogg_19/n_data.csv
Drop from 2952 to 2948 numerical pos rows
Loading: ./data/isogg_19/p_data.csv
Drop from 300 to 300 numer

## Call Individual Haplogroup
(change Name here)

In [66]:
punic_df = pd.read_csv("../punic_aDNA/data/males_feb20.csv")
punic_df.iloc[:,0].values

array(['I12517', 'I18193', 'I18199', 'I18202', 'I18194', 'I18201',
       'I18189'], dtype=object)

In [70]:
iid="I7259"
haplogroup = "j"

path_counts = f"./output/{iid}.txt"
df_counts = load_counts(path_counts)
df_isogg = dict_y_dfs[haplogroup]
df_call, _, _, _ = call_ind(df_counts, df_isogg, output=True, 
                            full=False)

Total rows of calls: 32670
Markers found in Database 814
Flipping 1 SNPs for Alt Allele
Markers witch calls 40
Frac. of Database SNPs covered: 0.049140
Markers derived: 10


In [72]:
df_call[-50:]

,snp,chr,pos,ref_all,alt_all,iid,ref,alt,Name,Haplogroup,Other Names in Tree,rs #,Build 38 #,Mutation info
108,snp_24_7065239,24,7065239,T,C,I7259,0,1,PF4505,J,YSC0000064,rs373806452,7197198,T->C
327,snp_24_9815201,24,9815201,T,C,I7259,0,1,PF4521,J,NaN,rs368797174,9977592,T->C
868,rs17222412,24,17495914,G,A,I7259,0,1,F2502,J,PF4564,rs17222412,15384034,G->A
262,snp_24_8432768,24,8432768,G,A,I7259,0,1,Y1422.1,J2b,Z8214.1,rs369289787,8564727,G->A
286,snp_24_8527247,24,8527247,G,A,I7259,0,1,Z2483,J2b,NaN,rs376725687,8659206,G->A
569,snp_24_15610489,24,15610489,G,A,I7259,0,1,CTS4359,J2b,PF7293; Z2417,rs375442273,13498609,G->A
604,snp_24_15740097,24,15740097,C,A,I7259,0,1,Z537,J2b,NaN,rs373683386,13628217,C->A
933,snp_24_17930928,24,17930928,T,C,I7259,0,1,CTS8314,J2b,PF7287; Z2491,rs372114377,15819048,T->C
501,rs8179022,24,15018459,G,A,I7259,0,1,M241,J2b2a,NaN,rs8179022,12906548,G->A
585,snp_24_15635257,24,15635257,C,T,I7259,0,1,AM01362,J2b2a~,CTS4412; Z2390,rs370252923,13523377,C->T


In [82]:
len(pull_clade(df_call, "R1b1b", exact=True))

37

# Run all Haplogroups

In [ ]:
%%time
iid="I16339"
for h in haplogroups:

#print(f"\nCalling {haplogroup}")
    print(f"\nDoing Haplogroup: {h}")
    path_counts = f"./output/{iid}.txt"
    df_counts = load_counts(path_counts)
    df_isogg = dict_y_dfs[h]
    df_call, _, _, _ = call_ind(df_counts, df_isogg, output=True, 
                                full=False)

# Area 51

In [8]:
df_t = load_counts("./output/I0410.txt")

In [20]:
np.sum(df_t["ref"]>0)

0

In [62]:
idx = df_isogg["Haplogroup"]=="R1b1a1b1b"
np.sum(idx)

15